PLL (Phase Locked Loop): control system that generates an output signal that has the phase related to the phase of an input signal. <br>
used in signal demodulation, recovering signal from a noisy communication channel generate a stable frequency at multiples of an input frequency (frequency synthesis), or distribute precisely timed clock pulses in digital logic circuits such as microprocessors. <br>

Phase detector - LP filter - Voltage controlled oscillator - feedback path



In [111]:
import math
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy import signal
from scipy.fft import fft, fftfreq

from ipywidgets import interact
import ipywidgets as widgets

In [143]:
class PLL(object):
    def __init__(self, f0, D, k, w0, T):
        """
        initialize the state of PLL
        """

        self.f0 = f0 
        self.T = T
        self.k = k
        self.D = D
        self.w0 = w0

        tau1 = k/(w0**2)
        tau2 = 2*D/w0 - 1/k

        self.a1 = -(T-2*tau1)/(T+2*tau1)
        self.b0 = (T+2*tau2)/(T+2*tau1)
        self.b1 = (T-2*tau2)/(T+2*tau1)

        self.sin_table = np.sin(2*np.pi*np.linspace(0,1, 1024))


        self.acc = 0
    
    def pll(self, x):
        """
        x : input function
        """

        z = np.zeros(x.size)
        v = np.zeros(x.size)
        y = np.zeros(x.size)
        #print(x)
        #print(z)
        for n in range(1, x.size):
            z[n] = x[n]*y[n-1]
            #print(z[n])
            v[n] = self.a1 * v[n-1] + self.b0 * z[n] + self.b1*z[n-1]
            #print(v[n])
            self.acc = self.acc +self.f0 - (self.k*v[n]/(2*np.pi))
            self.acc = self.acc- np.floor(self.acc)
            #print(np.floor(self.acc*1024))
            
            y[n] = self.sin_table[np.int(np.floor(1024*self.acc)+1)]
            #print(y[n])
        return y





        

In [144]:
Nb = 10
Ns = 100
f0 = 0.1
k = 1
D = 1
w0 = 2*np.pi/100
T= 1


state_pll = PLL(f0, D, k, w0, T)

t = np.linspace(0,1,8000)
x = np.cos(2*np.pi*800*t)
#print(x)
x = x[:1000]
xb = np.reshape(x,(Nb, Ns))

yb = np.zeros((Nb,Ns))
for j in range(Nb):
    yb[j] = state_pll.pll(xb[j])
yb = np.reshape(yb, Nb*Ns)


fig = go.Figure()
fig.add_trace(go.Scatter(y = x))
fig.add_trace(go.Scatter(y = yb))
fig

1000